# Implementing the Least Squares Classifier for MNIST dataset
### Dan Vu

In this project we will first implement a **binary least squares classifier** which will then be used as the building block for a **one vs rest multi-class classifier** and a **one vs one multi-class classifier** 

## Preparing our python environment

In [1]:
import numpy as np

import matplotlib.pyplot as plt

from scipy.io import loadmat

import pandas as pd

#allows pd.DataFrames to be printed without truncation
pd.options.display.max_columns = None 
pd.options.display.max_rows = None

## Defining some functions that will help us down the line

`show_img_lbl` displays the image (handwritten digit) and label 

`show_DF` will display the dataframe of a given array/matrix

In [2]:
def show_img_lbl(X, Y, nth):
    
    plt.figure()
    plt.imshow(X[nth,:].reshape(28,28),cmap='binary')
    print(Y[nth])

In [3]:
def show_DF(X):
    return pd.DataFrame(X).head(1000)

## These functions will be used to build the binary classifier

In [4]:
def preprocess(X,testX):
    
    newX = X
    newtestX = testX
    
    pixel_counter = np.zeros((1,784))
    
    for image_ind in range(len(X)):
        for pixel_ind in range(len(X[image_ind])):
            if X[image_ind,pixel_ind] != 0:
                pixel_counter[0,pixel_ind]=pixel_counter[0,pixel_ind]+1
                
    
    deletelist=[]
    for pixel_ind in range(len(X[0])):
        if pixel_counter[0,pixel_ind] < 600:
            deletelist.append(pixel_ind)
            
            
    newX = np.delete(newX, deletelist, axis=1)
    newtestX = np.delete(newtestX, deletelist, axis=1)
    
            
        
    return newX,newtestX

In [5]:
def get_binary_Y(Y, num):
    binary_Y=np.zeros(np.shape(Y), dtype=int)
    for i in range(np.size(Y)):
        if Y[i,0] == num:
            binary_Y[i,0]=1
        else:
            binary_Y[i,0]=-1
        
    return binary_Y

In [6]:
def getA(X):
    A=X
    A=A/255
    A = np.insert(A,0,1,axis=1)
    
    return A

In [7]:
def getBeta(A,Y):
    try:
        ATA=(A.T@A)
    #     print(ATA.shape) # 494,494
        ATA_inv=np.linalg.inv(ATA)
    #     print(ATA_inv.shape) # 494,494
        ATA_inv_AT=ATA_inv @ A.T
    #     print(ATA_inv_AT.shape) # 494,60000
        beta = ATA_inv_AT @ Y # 494,60000 @ 1,60000
    except:
        AT = A.T
        AAT = A @ A.T
        AAT_inv = np.linalg.inv(AAT)
        AT_AAT_inv = AT @ AAT_inv
        beta = AT_AAT_inv @ Y

    return beta

In [58]:
def testBeta(beta,ptestX):
    try:
        yhat=np.sign(beta.T @ ptestX.T)
    except:
        print(beta.shape,ptestX.shape)
        yhat=np.sign(beta @ ptestX.T)
    return yhat

In [9]:
def check_binaryClassifier(yhat, btestY):
    
    yhat=yhat.T
    btestY=btestY.T
    
    nfp,nfn,ntp,ntn=0,0,0,0
    for index in range(len(yhat)):
        if yhat[index] == btestY.T[index]:
            if yhat[index]==1:
                ntp=ntp+1
            else: 
                ntn=ntn+1
        elif yhat[index]!=btestY.T[index]:
            if yhat[index]==1:
                nfp=nfp+1
            else:
                nfn=nfn+1
                
    return nfp,nfn,ntp,ntn,
    

In [10]:
def generate_confusion(nfp,nfn,ntp,ntn):
    error_rate = (nfn+nfp)/(nfn+nfp+ntp+ntn)
    
    confusion = np.array([[ntp, nfn, ntp+nfn],[nfp, ntn, nfp+ntn]])
    return confusion, error_rate

## Binary Classifier

In [11]:
def binaryClassifier(ptrainX,btrainY):
    
    A=getA(ptrainX)
    beta=getBeta(A,btrainY)
    
    return beta

In [12]:
def test_binaryClassifier(ptestX, beta):
    
    testA=getA(ptestX)
    
    yhat=testBeta(beta,testA)
    
    return yhat

In [13]:
def test_binaryClassifier2(ptestX, beta): #without the signed function
    
    testA=getA(ptestX)
    
    ytilda=beta.T @ testA.T
    
    return ytilda

## Now that Binary Classifier is complete, we can build the multi-class classifiers

First, we will load in and prepare our data:

`prep()` will return trainY and testY in binary format (1,-1) with the appropriate given digit


 `preprocess()` will filter out all pixels EXCEPT for the ones that have a non zero value more than 600x

In [14]:
def prep(trainY,testY, num):
    btrainY=get_binary_Y(trainY, num)
    btestY=get_binary_Y(testY, num)
    
    return btrainY,btestY

In [15]:
data = loadmat("mnist.mat")
trainX = data['trainX']
trainY = data['trainY'].transpose()
testX = data['testX']
testY = data['testY'].transpose()

ptrainX,ptestX=preprocess(trainX,testX)

## One vs All (One vs Rest)

In [16]:
def onevsall(ptrainX,trainY,ptestX,testY):
    
    BET=np.array([[]])
    
    for num in range(10):
        btrainY,btestY = prep(trainY,testY,num)
        beta=binaryClassifier(ptrainX,btrainY)
        BET=np.append(BET,beta)
        
    BET=BET.reshape(10,beta.shape[0])
        
    return BET

In [34]:
def test_onevsall(trainY,ptestX,testY,BET):
    
    YTILDA=np.array([])
    
    for num in range(10):
        btrainY,btestY = prep(trainY,testY,num)
        ytilda=test_binaryClassifier2(ptestX,BET[num,:])
        YTILDA=np.append(YTILDA,ytilda)
        
    YTILDA=YTILDA.reshape(10,ptestX.shape[0])
        
    return YTILDA
        

We call the `onevsall()` which will return BET, an array of beta matrices or weights 

In [18]:
BET = onevsall(ptrainX,trainY,ptestX,testY)

then, we apply the testX dataset to our trained BET model by calling `test_onevsall()` to see how it performs

In [19]:
YTILDA=test_onevsall(trainY,ptestX,testY,BET)

we then find the argmax of YTILDA (ftilda(x)) to see which digit our model predicted from the testX dataset 

In [38]:
def getArgMax(YTilda):
    
    YHAT=np.argmax(YTilda, axis=0)
    
    YHAT=YHAT.reshape(YTilda.shape[1],1)
    
    return YHAT

In [21]:
YHAT=getArgMax(YTILDA)

this next function checks the performance of our model

In [31]:
def check_onevsall(YHAT,testY):
    
    confusion=np.zeros((10,10))
    
    for index in range(YHAT.shape[0]):
        confusion[testY[index],YHAT[index]]+=1
    
    error=1-np.trace(confusion)/(YHAT.shape[0])
    
    return error,confusion

In [32]:
erro,conff=check_onevsall(YHAT,testY)

## Confusion Matrix and Error Rate(Test Set)

Confusion Matrix:
- horizontal axis = predictions
- vertical axis = reference digit

In [33]:
print('Error rate: {}'.format(erro))
print('Percent correct: {}'.format((1-erro)*100))
show_DF(conff)

Error rate: 0.13929999999999998
Percent correct: 86.07000000000001


,0,1,2,3,4,5,6,7,8,9
0,944.0,0.0,1.0,2.0,2.0,8.0,13.0,2.0,7.0,1.0
1,0.0,1107.0,2.0,2.0,3.0,1.0,5.0,1.0,14.0,0.0
2,18.0,54.0,815.0,26.0,16.0,0.0,38.0,22.0,39.0,4.0
3,4.0,18.0,22.0,884.0,5.0,16.0,10.0,22.0,20.0,9.0
4,0.0,22.0,6.0,0.0,883.0,3.0,9.0,1.0,12.0,46.0
5,24.0,19.0,3.0,74.0,24.0,656.0,24.0,13.0,38.0,17.0
6,17.0,9.0,10.0,0.0,22.0,17.0,876.0,0.0,7.0,0.0
7,5.0,43.0,14.0,6.0,25.0,1.0,1.0,883.0,1.0,49.0
8,14.0,48.0,11.0,31.0,26.0,40.0,17.0,13.0,756.0,18.0
9,16.0,10.0,3.0,17.0,80.0,0.0,1.0,75.0,4.0,803.0


## Confusion Matrix and Error Rate(Training Set)

Confusion Matrix:
- horizontal axis = predictions
- vertical axis = reference digit

here, we show the training error of our model

In [35]:
YTILDA2=test_onevsall(trainY,ptrainX,testY,BET)
YHAT2=getArgMax(YTILDA2)

In [40]:
err2,conff2=check_onevsall(YHAT2,trainY)

In [41]:
print('Error rate: {}'.format(err2))
print('Percent correct: {}'.format((1-err2)*100))
show_DF(conff2)

Error rate: 0.14446666666666663
Percent correct: 85.55333333333334


,0,1,2,3,4,5,6,7,8,9
0,5669.0,8.0,21.0,19.0,25.0,46.0,65.0,4.0,60.0,6.0
1,2.0,6543.0,36.0,17.0,20.0,30.0,14.0,14.0,60.0,6.0
2,99.0,278.0,4757.0,153.0,116.0,17.0,234.0,92.0,190.0,22.0
3,38.0,172.0,174.0,5150.0,31.0,122.0,59.0,122.0,135.0,128.0
4,13.0,104.0,41.0,5.0,5189.0,52.0,45.0,24.0,60.0,309.0
5,164.0,94.0,30.0,448.0,103.0,3974.0,185.0,44.0,237.0,142.0
6,104.0,78.0,77.0,2.0,64.0,106.0,5448.0,0.0,36.0,3.0
7,55.0,191.0,36.0,48.0,165.0,9.0,4.0,5443.0,13.0,301.0
8,69.0,492.0,64.0,225.0,102.0,220.0,64.0,21.0,4417.0,177.0
9,67.0,66.0,26.0,115.0,365.0,12.0,4.0,513.0,39.0,4742.0


# One vs One multi-class classifier

we will now implement 1v1 classifier

`label_ovo()` will translate the labels into binary (1,-1) for a specific given combination out of (10 choose 2)  

In [43]:
def label_ovo(newTrainY, combo):
    
    binary=np.zeros(newTrainY.shape)
    
    for index in range(newTrainY.shape[0]):
        if newTrainY[index]==combo[0]:
            binary[index]=1
        else:
            binary[index]=-1
    
    return binary

In [75]:
def onevsone(ptrainX,trainY):
    
    from itertools import combinations
    
    combos=np.array(list(combinations([j for j in range(10)], 2)))
    new_ptrainX=ptrainX
    
    BET=np.array([[]])
    
    for combo in combos:
        deletelist=[]
        for index in range(trainY.shape[0]):
            if trainY[index] not in combo:
                deletelist.append(index)    
                
        new_trainY = np.delete(trainY, deletelist, axis=0)
        newer_trainY = label_ovo(new_trainY,combo)
        
        new_ptrainX = np.delete(ptrainX, deletelist, axis=0)
        
        beta=binaryClassifier(new_ptrainX,newer_trainY)
        BET=np.append(BET,beta)
    
    BET=BET.reshape(combos.shape[0],beta.shape[0])
        
    
    
    return BET

Here, we train the 1v1 model. It will take longer than the one vs all model because we are training (10 choose 2)=45 models instead of 10

In [76]:
BETT = onevsone(ptrainX,trainY)

In [115]:
def count_votes(votes):
    from itertools import combinations
    import random
    
    combos=np.array(list(combinations([j for j in range(10)], 2)))
    
#     counts = {[j for j in range(10)][i]:([0]*10)[i] for i in range(10)}
    
    WINS=np.array([],dtype=int)
    
    
    for image_index in range(votes.shape[1]):
        counts = {[j for j in range(10)][i]:([0]*10)[i] for i in range(10)}
        for group_index in range(votes.shape[0]):
            if votes[group_index,image_index]==1:
                counts[combos[group_index,0]]+=1
            elif votes[group_index,image_index]==-1:
                counts[combos[group_index,1]]+=1
        winner= [k for k, v in counts.items() if v == max(counts.values())]
        WINS = np.append(WINS,winner)
        if len(winner)>1:
            tie_winner = random.choice(winner) #Ties are decided at random
            WINS = np.append(WINS,tie_winner)
        else:
            WINS = np.append(WINS,winner)
    WINS = WINS.reshape(votes.shape[1],1)
            
    return WINS, 

In [112]:
def test_onevsone(ptestX, BETT):
    votes=np.array([])
    
    for group in range(BETT.shape[0]):
        vote = test_binaryClassifier(ptestX,BETT[group,:])
        votes = np.append(votes,vote)
        
    votes=votes.reshape(45,ptestX.shape[0])

    return votes

In [114]:
print(votes.shape)

(45, 10000)


Now, we will test our 1v1 model and then count the votes that each digit has received. The prediction output is placed into WINS


In [106]:
votes = test_onevsone(ptestX,BETT)

In [107]:
WINS=count_votes(votes)

In [108]:
def check_ovo(WINS,testY):
    confusion=np.zeros((10,10))
    
    for index in range(WINS.shape[0]):
        confusion[testY[index],WINS[index]]+=1
    
    error=1-np.trace(confusion)/(WINS.shape[0])
    
    return error,confusion

In [109]:
errovo,confovo=check_ovo(WINS,testY)

## Confusion Matrix and Error Rate 1v1(test set)

Confusion Matrix:
- horizontal axis = predictions
- vertical axis = reference digit


In [110]:
print('Error rate: {}'.format(errovo))
print('Percent correct: {}'.format((1-errovo)*100))
show_DF(confovo)

Error rate: 0.40780000000000005
Percent correct: 59.21999999999999


,0,1,2,3,4,5,6,7,8,9
0,823.0,0.0,75.0,1.0,0.0,2.0,73.0,3.0,1.0,2.0
1,1.0,91.0,533.0,329.0,1.0,11.0,36.0,14.0,119.0,0.0
2,121.0,3.0,714.0,60.0,25.0,3.0,69.0,0.0,31.0,6.0
3,6.0,1.0,19.0,934.0,0.0,11.0,7.0,2.0,24.0,6.0
4,2.0,0.0,5.0,2.0,902.0,1.0,51.0,3.0,2.0,14.0
5,32.0,2.0,10.0,180.0,16.0,300.0,78.0,14.0,232.0,28.0
6,54.0,3.0,74.0,1.0,126.0,14.0,676.0,0.0,9.0,1.0
7,13.0,3.0,49.0,30.0,53.0,14.0,10.0,413.0,16.0,427.0
8,21.0,1.0,45.0,94.0,15.0,95.0,42.0,15.0,611.0,35.0
9,6.0,3.0,2.0,22.0,269.0,12.0,10.0,185.0,42.0,458.0


In [116]:
votes2 = test_onevsone(ptrainX,BETT)
WINS2=count_votes(votes2)
errovo2,confovo2=check_ovo(WINS2,trainY)

## Confusion Matrix and Error Rate 1v1(training set)

Confusion Matrix:
- horizontal axis = predictions
- vertical axis = reference digit

In [117]:
print('Error rate: {}'.format(errovo2))
print('Percent correct: {}'.format((1-errovo2)*100))
show_DF(confovo2)

Error rate: 0.40495000000000003
Percent correct: 59.504999999999995


,0,1,2,3,4,5,6,7,8,9
0,4983.0,0.0,406.0,6.0,12.0,9.0,482.0,7.0,14.0,4.0
1,8.0,460.0,3396.0,1849.0,14.0,109.0,174.0,57.0,660.0,15.0
2,778.0,23.0,4020.0,271.0,109.0,20.0,517.0,18.0,173.0,29.0
3,45.0,5.0,125.0,5627.0,12.0,80.0,37.0,27.0,131.0,42.0
4,17.0,3.0,16.0,6.0,5390.0,6.0,290.0,15.0,17.0,82.0
5,244.0,9.0,98.0,982.0,96.0,1894.0,494.0,121.0,1256.0,227.0
6,269.0,21.0,571.0,4.0,639.0,128.0,4213.0,0.0,71.0,2.0
7,98.0,7.0,194.0,157.0,375.0,69.0,66.0,2680.0,149.0,2470.0
8,101.0,12.0,282.0,714.0,67.0,543.0,214.0,49.0,3703.0,166.0
9,29.0,1.0,30.0,117.0,1446.0,58.0,73.0,1222.0,240.0,2733.0


## Evaluating Performances of Both Classifiers

### 1vsALL:
- Training Error Rate: 0.144
- Testing Error Rate: 0.139

### 1vs1
- Training Error: 0.404
- Testing Error: 0.407



**In both implementations**, the generalize quite well on the Testing dataset and maintain a similar Error rate. This is favorable since we do not want to over parameterize to the training set. This is likely the result of filtering out the pixels that were not significant before training as well as adding the biased term (column of ones) to the training set. 

**Note** For 1vs1 implementation, there were many occurences of a tie. Ties were decided at random, thus, the error rates for this implementation is heavily skewed and inaccurate. Ties often happened when 2 numbers had similar structure such as 3&5 or 3&8 or 4&9. 

The ties are an interesting note because it also has some relationship with the misclassified digits from the 1vsALL implementation where these digits were often mistaken for each other. 